<a href="https://colab.research.google.com/github/jorgemunozl/vla-test/blob/main/pi05-test/nine_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nine Test

This breaks the continuity that we have been following. But it's important because for debug properly we need a proper batch. The eight test continues being the important one.

In [2]:
from huggingface_hub import login
login()

In [3]:
!

Cloning into 'XHUMAN'...
remote: Enumerating objects: 2060, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 2060 (delta 166), reused 171 (delta 95), pack-reused 1794 (from 2)
Receiving objects: 100% (2060/2060), 7.81 MiB | 5.95 MiB/s, done.
Resolving deltas: 100% (1309/1309), done.


In [4]:
%cd XHUMAN

/content/XHUMAN


In [5]:
!uv pip install -e .[pi]

Using Python 3.12.12 environment at: /usr
Resolved 215 packages in 30.52s
Prepared 46 packages in 22.36s
Uninstalled 15 packages in 980ms
Installed 46 packages in 214ms
 + async-lru==2.0.5
 + av==15.1.0
 + comm==0.2.3
 + deepdiff==8.6.1
 - diffusers==0.36.0
 + diffusers==0.35.2
 + draccus==0.10.0
 + evdev==1.9.2
 + faker==40.1.0
 + feetech-servo-sdk==1.0.0
 - huggingface-hub==0.36.0
 + huggingface-hub==0.35.3
 + inquirerpy==0.3.4
 - ipykernel==6.17.1
 + ipykernel==7.1.0
 + jedi==0.19.2
 + json5==0.13.0
 + jsonlines==4.0.0
 + jupyter==1.1.1
 - jupyter-client==7.4.9
 + jupyter-client==8.8.0
 + jupyter-lsp==2.3.0
 + jupyterlab==4.5.2
 + jupyterlab-server==2.28.0
 + lerobot==0.4.3 (from git+https://github.com/huggingface/lerobot.git@15724826dd5b18c18656151a02a24ce5fd690c46)
 + mergedeep==1.3.4
 + mypy-extensions==1.1.0
 - notebook==6.5.7
 + notebook==7.5.2
 - nvidia-cudnn-cu12==9.10.2.21
 + nvidia-cudnn-cu12==9.5.1.17
 - nvidia-cusparselt-cu12==0.7.1
 + nvidia-cusparselt-cu12==0.6.3
 - nvi

In [7]:
import torch
from xhuman.configs.default import LerobotDatasetConfig

dataset_config = LerobotDatasetConfig(
    repo_id="NONHUMAN-RESEARCH/TEST_RECORD_ANNOTATIONS",
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
from xhuman.policies.pi05.configuration_pi05 import PI05Config

policy_config = PI05Config(
    device=device,
    repo_id="none/none"
)

In [12]:
from xhuman.configs.train import TrainPipelineConfigXHUMAN

cfg = TrainPipelineConfigXHUMAN(
    dataset=dataset_config,
    policy=policy_config,
    batch_size=50,
    num_workers=4,
    split_ratio=0.8,
)

cfg.validate()

In [13]:
from accelerate import Accelerator
from accelerate.utils import DistributedDataParallelKwargs

ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)
accelerator = Accelerator(step_scheduler_with_optimizer=False, kwargs_handlers=[ddp_kwargs])

In [17]:
from xhuman.datasets.factory import make_dataset_xhuman
from xhuman.datasets.utils import split_train_eval_episodes

dataset = make_dataset_xhuman(cfg)
episodes = list(range(dataset.meta.total_episodes))
train_episodes, _ = split_train_eval_episodes(episodes, split_ratio=cfg.split_ratio, seed=42)
if 30 in train_episodes:
    train_episodes.remove(30)

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

In [18]:
from xhuman.policies.factory import make_xhuman_policy

policy = make_xhuman_policy(cfg=cfg.policy, ds_meta=dataset.meta)

processor_config.json:   0%|          | 0.00/253 [00:00<?, ?B/s]

processing_action_tokenizer.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/physical-intelligence/fast:
- processing_action_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/322 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [20]:
processor_kwargs = {"dataset_stats": dataset.meta.stats}
if cfg.policy.pretrained_path is not None:
    processor_kwargs["preprocessor_overrides"] = {
        "device_processor": {"device": device.type},
        "normalizer_processor": {
            "stats": dataset.meta.stats,
            "features": {**policy.config.input_features, **policy.config.output_features},
            "norm_map": policy.config.normalization_mapping,
        },
    }


In [23]:
cfg.policy

PI05Config(n_obs_steps=1, input_features={'observation.state': PolicyFeature(type=<FeatureType.STATE: 'STATE'>, shape=(14,)), 'observation.images.left': PolicyFeature(type=<FeatureType.VISUAL: 'VISUAL'>, shape=(3, 376, 672)), 'observation.images.top': PolicyFeature(type=<FeatureType.VISUAL: 'VISUAL'>, shape=(3, 376, 672)), 'observation.images.right': PolicyFeature(type=<FeatureType.VISUAL: 'VISUAL'>, shape=(3, 376, 672))}, output_features={'action': PolicyFeature(type=<FeatureType.ACTION: 'ACTION'>, shape=(14,))}, device=device(type='cuda'), use_amp=False, use_peft=False, push_to_hub=True, repo_id='none/none', private=None, tags=None, license=None, pretrained_path=None, alpha=10.0, max_decoding_steps=100, temperature=0.1, eos_token_id=1, paligemma_variant='gemma_2b', action_expert_variant='gemma_300m', dtype='float32', chunk_size=50, n_action_steps=50, max_state_dim=32, max_action_dim=32, num_inference_steps=10, time_sampling_beta_alpha=1.5, time_sampling_beta_beta=1.0, time_sampling_s

In [27]:
from xhuman.policies.factory import (make_xhuman_pre_post_processors)

preprocessor, _ = make_xhuman_pre_post_processors(
    policy_cfg=cfg.policy,
    pretrained_path=None,
    **processor_kwargs,
)

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

In [28]:
from xhuman.datasets.factory import make_dataset_xhuman

cfg.dataset.episodes = train_episodes
dataset = make_dataset_xhuman(cfg)

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

In [29]:
from lerobot.datasets.sampler import EpisodeAwareSampler

# Crear dataloader
if hasattr(cfg.policy, "drop_n_last_frames"):
    sampler = EpisodeAwareSampler(
        dataset.meta.episodes["dataset_from_index"],
        dataset.meta.episodes["dataset_to_index"],
        drop_n_last_frames=cfg.policy.drop_n_last_frames,
        shuffle=True,
    )
else:
    sampler = None


In [30]:
dataloader = torch.utils.data.DataLoader(
    dataset,
    num_workers=cfg.num_workers,
    batch_size=cfg.batch_size,
    shuffle=(sampler is None) and not cfg.dataset.streaming,
    sampler=sampler,
    pin_memory=device.type == "cuda",
    drop_last=False,
    prefetch_factor=2 if cfg.num_workers > 0 else None,
)

In [33]:
from lerobot.datasets.utils import cycle
dataloader = accelerator.prepare(dataloader)
dl_iter = cycle(dataloader)

In [39]:
# Fix: Initialize sampler with correct RELATIVE indices mapped from valid absolute ranges
import torch
from lerobot.datasets.sampler import EpisodeAwareSampler
from lerobot.datasets.utils import cycle

if hasattr(cfg.policy, "drop_n_last_frames"):
    # 1. Identify valid episodes in the current dataset
    valid_episodes = set(dataset.episodes)

    # 2. Access the absolute -> relative mapping
    # This tells us which absolute frames effectively exist at which relative row
    abs_to_rel = dataset._absolute_to_relative_idx

    # 3. Build RELATIVE ranges for the sampler
    meta_eps = dataset.meta.episodes
    ids = meta_eps["episode_index"]
    froms = meta_eps["dataset_from_index"]
    tos = meta_eps["dataset_to_index"]

    sampler_starts = []
    sampler_ends = []

    print("Constructing RELATIVE sampler ranges...")
    for i, ep_id in enumerate(ids):
        if ep_id.item() in valid_episodes:
            # Metadata Absolute Range
            start_abs = froms[i].item()
            end_abs = tos[i].item()

            # Find which of these absolute indices exist in our filtered dataset
            # and map them to relative indices
            valid_rel_indices = [
                abs_to_rel[a] for a in range(start_abs, end_abs)
                if a in abs_to_rel
            ]

            if valid_rel_indices:
                # We assume the relative indices for an episode are contiguous
                # (Standard LerobotDataset behavior)
                rel_start = min(valid_rel_indices)
                rel_end = max(valid_rel_indices) + 1

                sampler_starts.append(rel_start)
                sampler_ends.append(rel_end)
                print(f"  Ep {ep_id.item()}: Abs [{start_abs}, {end_abs}) -> Rel [{rel_start}, {rel_end})")
            else:
                print(f"  Ep {ep_id.item()}: Abs [{start_abs}, {end_abs}) -> NO DATA LOADED")

    # 4. Create sampler with these VALID RELATIVE indices
    sampler = EpisodeAwareSampler(
        sampler_starts,
        sampler_ends,
        drop_n_last_frames=cfg.policy.drop_n_last_frames,
        shuffle=True,
    )
else:
    sampler = None

# Re-create DataLoader
dataloader = torch.utils.data.DataLoader(
    dataset,
    num_workers=cfg.num_workers,
    batch_size=cfg.batch_size,
    shuffle=(sampler is None) and not cfg.dataset.streaming,
    sampler=sampler,
    pin_memory=device.type == "cuda",
    drop_last=False,
    prefetch_factor=2 if cfg.num_workers > 0 else None,
)

# Prepare with accelerator and cycle
dataloader = accelerator.prepare(dataloader)
dl_iter = cycle(dataloader)

# ---------------------------------------------------------
# Inspection
# ---------------------------------------------------------
print("=" * 80)
print("RAW BATCH (before preprocessor) - FIXED SAMPLER (RELATIVE MAPPED):")
print("=" * 80)

# Get a fresh batch
raw_batch = next(dl_iter)
print(f"\nRaw batch keys: {list(raw_batch.keys())}\n")

for key, value in raw_batch.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: shape={value.shape}, dtype={value.dtype}")
    elif isinstance(value, (list, tuple)):
        print(f"  {key}: {type(value).__name__} of length {len(value)}")
        if len(value) > 0 and isinstance(value[0], torch.Tensor):
            print(f"    First element shape: {value[0].shape}")
    else:
        print(f"  {key}: {type(value).__name__} = {value}")

print("\n" + "=" * 80)

RAW BATCH (before preprocessor) - FIXED SAMPLER (RELATIVE MAPPED):


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/content/XHUMAN/xhuman/datasets/xhuman_dataset.py", line 151, in __getitem__
    query_result = self._query_hf_dataset(query_indices)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lerobot/datasets/lerobot_dataset.py", line 993, in _query_hf_dataset
    else [self._absolute_to_relative_idx[idx] for idx in q_idx]
          ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^
KeyError: 1014


In [36]:
import inspect
from xhuman.datasets.xhuman_dataset import XHumanDataset

# 1. Print the source of __getitem__ to understand how it handles indices
print("=== XHumanDataset.__getitem__ Source ===")
try:
    print(inspect.getsource(XHumanDataset.__getitem__))
except Exception as e:
    print(f"Could not get source: {e}")

# 2. Inspect dataset internal mappings
print("\n=== Dataset Internal Mappings ===")
if hasattr(dataset, "indices"):
    print(f"dataset.indices type: {type(dataset.indices)}")
    if hasattr(dataset.indices, "__len__"):
        print(f"dataset.indices length: {len(dataset.indices)}")
        print(f"First 10 indices: {dataset.indices[:10]}")

if hasattr(dataset, "episode_data_index"):
    print(f"dataset.episode_data_index keys: {dataset.episode_data_index.keys()}")

# Check if there is an absolute_to_relative_idx map
if hasattr(dataset, "_absolute_to_relative_idx"):
    print(f"_absolute_to_relative_idx size: {len(dataset._absolute_to_relative_idx)}")
    # Print a few sample keys
    print(f"Sample keys: {list(dataset._absolute_to_relative_idx.keys())[:5]}")

=== XHumanDataset.__getitem__ Source ===
    def __getitem__(self, idx) -> dict:
        # Ensure dataset is loaded when we actually need to read from it
        self._ensure_hf_dataset_loaded()
        item = self.hf_dataset[idx]
        ep_idx = item["episode_index"].item()

        query_indices = None
        if self.delta_indices is not None:
            query_indices, padding = self._get_query_indices(idx, ep_idx)
            query_result = self._query_hf_dataset(query_indices)
            item = {**item, **padding}
            for key, val in query_result.items():
                item[key] = val

        if len(self.meta.video_keys) > 0:
            current_ts = item["timestamp"].item()
            query_timestamps = self._get_query_timestamps(current_ts, query_indices)
            video_frames = self._query_videos(query_timestamps, ep_idx)
            item = {**video_frames, **item}

        if self.image_transforms is not None:
            image_keys = self.meta.camera_keys
  

## Understanding `cfg.steps` vs Epochs

In `train_val_pi05.py`, the training loop uses:

```python
for _ in range(step, cfg.steps):
    batch = next(dl_iter)  # Get next batch from infinite iterator
    batch = preprocessor(batch)
    # ... training step ...
    step += 1
```

**Key points:**

1. **`cfg.steps`** = Total number of training steps (gradient updates), NOT epochs
2. **`cycle(dataloader)`** creates an infinite iterator that automatically restarts when exhausted
3. This approach is common in RL/robotics because:
   - You care more about number of gradient updates than epochs
   - Datasets can be very large, so epochs might take too long
   - You can easily control training duration with a step count

**Example calculation:**
- Dataset: 10,000 samples
- Batch size: 50
- Batches per epoch: 10,000 / 50 = 200 batches
- If `cfg.steps = 1000`:
  - We do 1000 gradient updates
  - This equals 1000 / 200 = 5 epochs worth of data
  - But we don't explicitly count epochs, just steps

In [ ]:
# Let's inspect the raw batch BEFORE preprocessing to see the original structure
print("=" * 80)
print("RAW BATCH (before preprocessor):")
print("=" * 80)

# Get a fresh batch before preprocessing
raw_batch = next(dl_iter)
print(f"\nRaw batch keys: {list(raw_batch.keys())}\n")

for key, value in raw_batch.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: shape={value.shape}, dtype={value.dtype}")
    elif isinstance(value, (list, tuple)):
        print(f"  {key}: {type(value).__name__} of length {len(value)}")
        if len(value) > 0 and isinstance(value[0], torch.Tensor):
            print(f"    First element shape: {value[0].shape}")
    else:
        print(f"  {key}: {type(value).__name__} = {value}")

print("\n" + "=" * 80)
print("COMPARISON:")
print("=" * 80)
print("Raw batch has the original dataset structure")
print("Processed batch (after preprocessor) has the structure expected by the policy")

In [ ]:
# Obtener batch procesado (igual que next(dl_iter) en train_val_pi05.py)
batch = next(dl_iter)
batch = preprocessor(batch)

In [ ]:
# El batch está listo para usar
print("Batch keys:", list(batch.keys()))
for key, value in batch.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: shape={value.shape}, dtype={value.dtype}")